<a href="https://colab.research.google.com/github/niklas-joh/6-CoderSwag/blob/master/Budget_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss, f1_score

Load classified 2022 dataset

In [ ]:
# Load the 2022 dataset
file_path_2022 = 'Budget2022.xlsx'  # Replace with your file path
budget_data_2022 = pd.read_excel(file_path_2022, header=6)  # Adjust header row if needed
budget_data_2022.head()


Preprocess the 2022 dataset

In [ ]:
# Preprocess the 2022 dataset
# Convert 'Date' to datetime and extract relevant parts
budget_data_2022['Date'] = pd.to_datetime(budget_data_2022['Date'])
budget_data_2022['Year'] = budget_data_2022['Date'].dt.year
budget_data_2022['Month'] = budget_data_2022['Date'].dt.month
budget_data_2022['Day'] = budget_data_2022['Date'].dt.day
budget_data_2022['Is_weekend'] = budget_data_2022['Date'].dt.weekday >= 5

# Convert 'Amount' to numeric
budget_data_2022['Amount'] = pd.to_numeric(budget_data_2022['Amount'], errors='coerce')

In [ ]:
# Check for missing values
missing_values = budget_data_2022.isnull().sum()
data_types = budget_data_2022.dtypes

missing_values, data_types

In [ ]:
# Display only rows where 'Sub-Category' is missing. Change column name to see other columns
missing_values = budget_data_2022[budget_data_2022['Sub-Category'].isna()]

# Displaying the result
missing_values

In [ ]:
# Handle missing values

# Fill missing values in 'Sub-Category' with corresponding 'Category' values
budget_data_2022['Sub-Category'] = budget_data_2022['Sub-Category'].fillna(budget_data_2022['Category'])

# Display only rows where 'Sub-Category' is missing. Change column name to see other columns
missing_values = budget_data_2022[budget_data_2022['Sub-Category'].isna()]

# Display the result
missing_values

In [ ]:
# Dropping rows where 'Type', 'Heading', or 'Category' is missing
budget_data_2022 = budget_data_2022.dropna(subset=['Type', 'Heading', 'Category'])
missing_values = budget_data_2022[budget_data_2022['Category'].isna()]

# Display the result
missing_values

In [ ]:
# Check for missing values
missing_values = budget_data_2022[budget_data_2022['Operation'].isna()]

# Display the result
missing_values

In [ ]:
# Apply TF-IDF to 'Operation', a text analysis that splits text into multiple columns
tfidf = TfidfVectorizer(max_features=100)
operation_tfidf = tfidf.fit_transform(budget_data_2022['Operation'])
operation_tfidf_df = pd.DataFrame(operation_tfidf.toarray(), columns=tfidf.get_feature_names_out())
operation_tfidf_df.isna()

In [ ]:
# Encode categorical variables and remove non-relevant columns
encoded_columns = pd.get_dummies(budget_data_2022[['Type', 'Heading', 'Category', 'Sub-Category']])
columns_to_drop = ['#','Operation', 'Date', 'Beneficiary Name', 'Beneficiary Account', 'Communication', 'Card','Countervalue', 'Currency2', 'City', 'Country']
budget_data_2022_processed = pd.concat([budget_data_2022, encoded_columns, operation_tfidf_df], axis=1).drop(columns_to_drop, axis=1)
budget_data_2022_processed

In [ ]:
# Check for missing values
# missing_values = budget_data_2022_processed.isnull().sum()

# missing_values

# Display only rows where 'Sub-Category' is missing. Change column name to see other columns
# missing_values = budget_data_2022_processed[budget_data_2022_processed['Type'].isna()]

# Display the result
# missing_values
budget_data_2022_processed = budget_data_2022_processed.dropna()
budget_data_2022_processed

In [ ]:
# Split the data into features and targets
target_columns = ['Type', 'Heading', 'Category', 'Sub-Category']  # Adjust as per your target labels

# Concatenate the target columns into one column separated by '|'
budget_data_2022_processed['Combined_Targets'] = budget_data_2022_processed[target_columns].apply(lambda x: '|'.join(x.astype(str)), axis=1)

# Now, your y values are in the 'Combined_Targets' column
y = budget_data_2022_processed['Combined_Targets']
combined_targets = ['Combined_Targets']

In [ ]:
X = budget_data_2022_processed.drop(target_columns, axis=1)
# y = budget_data_2022_processed[target_columns]
X.head()

In [ ]:
X = X.drop(combined_targets, axis=1)
X.head()

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()
#y_train.head()

In [ ]:
# Check for missing values
# X_train_missing_values = X_train.isnull().sum()

# X_train_missing_values

# Display only rows where 'Sub-Category' is missing. Change column name to see other columns
# X_train_missing_values_column = X_train[X_train['Amount'].isna()]

# Display the result
# X_train_missing_values_column
# X.iloc[1111:1118]
# y.isna()
# y.dropna()
# y.reset_index()
# y.isna()


In [ ]:
# Initialise the Random Forest Classifier and wrap in MultiOutputClassifier
rf_classifier = RandomForestClassifier(random_state=42)
# multi_output_classifer = MultiOutputClassifier(rf_classifier)

# Train the model
# multi_output_classifier.fit(X_train, y_train)
rf_classifier.fit(X_train, y_train)
# Evaluate the model
y_pred = rf_classifier.predict(X_val)

In [ ]:
# accuracy is not applicable for multi-label output, only on single-label output. Uncomment if needed.
accuracy = accuracy_score(y_val, y_pred)
# classification_report_results = classification_report(y_val, y_pred, target_names=combined_targets, zero_division=0)
print("Accuracy:", accuracy)
# print("Classification Report:\n", classification_report_results);

# Output the accuracy and classification report

# Evaluate the model using appropriate multi-label metrics
# Hamming Loss
# hamming_loss_val = hamming_loss(y_val, y_pred)

# F1 Score - calculate for each label and then find their unweighted mean
# f1_score_val = f1_score(y_val, y_pred, average='samples')

# print("Hamming Loss:", hamming_loss_val)
# print("F1 Score (Sample Average):", f1_score_val)

# For a detailed classification report
# print("Classification Report:\n", classification_report(y_val, y_pred, zero_division=0))


In [ ]:
y_pred

In [ ]:
# Export to Excel for manual compare

# Convert predictions and actual values to DataFrame
predictions_df = pd.DataFrame(y_pred, columns=['Pred_' + col for col in y_val.columns])
actual_df = pd.DataFrame(y_val.to_numpy(), columns=['Actual_' + col for col in y_val.columns], index=y_val.index)

# Combine both DataFrames
comparison_df = pd.concat([actual_df, predictions_df], axis=1)

# Write the DataFrame to an Excel file
comparison_df.to_excel("predictions_comparison.xlsx")
